# Modelo Regresión Logística Multivariable

Para más información sobre la implementación favor de consultar el reporte final del reto en la sección Selección, Configuración y Entrenamiento del Modelo. El reporte esta ubicado en el repositorio en la carpeta llamada Modelos, junto con las carpetas de cada modelo. El nombre del archivo es: ReporteFinal_Equipo7.pdf. También se puede consultar por medio del siguiente link: https://docs.google.com/document/d/199vDBFCTWJKEMpJC6qNKBOe6Gf4dxnn06aPI6NvIPmA/edit?usp=sharing

Elaborado por Marcelo Márquez Murillo - A01720588, para la clase TC3006C.101 del equipo 7

In [313]:
# Importamos las librerías necesarias
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

## Creamos clase de regresión logísitica

In [314]:
class LogisticRegression:
    def __init__(self, independent, dependent, num_iters, learning_rate):
        self.dependent = dependent
        self.n = len(dependent)
        # Agregar una columna de unos para que se multiple con el theta0
        self.independent = np.c_[np.ones(self.n), independent]
        self.num_iters = num_iters
        self.learning_rate = learning_rate
        # Crear lista de unos tomando en cuenta el número de variables independientes +1 
        self.theta = np.ones(independent.shape[1] + 1)
    
    def h(self, x_n):
        # Multiplicación de matriz de variables independientes con lista de theta
        # z = theta0*1 + theta1*xi1 + theta2*xi2
        z = np.dot(x_n, self.theta)
        return 1 / (1 + np.exp(-z))
    
    def fit(self):
        # Iniciar un bucle que se ejecutará durante 'num_iters' iteraciones
        for i in range(self.num_iters):
            
            # Utilizar el método 'h' para calcular las predicciones de todas las muestras
            # Esto se realiza utilizando la matriz completa de variables independientes
            predictions = self.h(self.independent)
            
            # Calcular el error (o delta) entre las predicciones y los valores reales (dependientes)
            # El resultado es un vector que contiene el error para cada muestra
            delta = predictions - self.dependent
            
            # Calcular el gradiente del error con respecto a los parámetros theta
            # Para ello, se realiza el producto punto entre la matriz traspuesta de las variables independientes
            # y el vector de errores, y luego se divide por el número total de muestras (self.n)
            gradient = np.dot(self.independent.T, delta) / self.n
            
            # Actualizar los valores de los parámetros theta utilizando el gradiente y la tasa de aprendizaje
            # La actualización se realiza en dirección opuesta al gradiente para minimizar la función de costo
            self.theta -= self.learning_rate * gradient
        
        # Imprimir los valores finales de los parámetros theta
        print(f"Thetas finales: {self.theta}")

    
    def predict(self, xi):
        # Renglón de X y le agregamos un 1 para theta0
        xi = np.c_[np.ones(xi.shape[0]), xi]
        prob = self.h(xi)
        return np.round(prob)
    
    @staticmethod
    def evaluate(x_test, y_test):
        vp = 0
        vn = 0
        fp = 0
        fn = 0
        
        # Conseguimos matriz de confusión
        n = len(x_test)
        for i in range(n):
            if y_test[i] == 1: 
                if x_test[i] == 1:
                    vp += 1
                else:
                    fp += 1
            else:
                if x_test[i] == 0:
                    vn += 1
                else:
                    fn += 1
        print(f"Model: vp - {vp}, vn - {vn}, fp - {fp}, fn - {fn}")
        
        # Conseguimos las métricas
        a = round((vp + vn) / (vp + vn + fp + fn), 2)
        p = round(vp / (vp + fp), 2)
        r = round(vp / (vp + fn), 2)
        f1 = round((2 * p * r) / (p + r), 2)
        print(f"Accuracy: {a}, Precision: {p}, Recall: {r}, F1 Score: {f1}")

## Configuración 1

In [315]:
# Conseguir los datos de la primera configuración
df1 = pd.read_csv("train_ff_split.csv")
# Conseguir las variables independientes
x = df1.drop("Survived", axis=1)
# Conseguir la variable dependiente
y = df1["Survived"].tolist()

df_test = pd.read_csv("train_transformed.csv")
test_x = df_test.drop("Survived", axis=1)
test_y = df_test["Survived"].tolist()

In [316]:
# Crear modelo
model1 = LogisticRegression(independent=x, dependent=y, num_iters=350000, learning_rate=0.05)
# Entrenamos al modelo
model1.fit()

Thetas finales: [ 19.22645254  -3.36456472  -0.21479313 -10.30627054  -0.79546938
   0.93186388  -0.58330492]


### Hacemos pruebas de la configuración 1 con una matriz de confusión

In [317]:
print("Configuración 1")
predictions = model1.predict(test_x)
model1.evaluate(predictions, test_y)

Configuración 1
Model: vp - 254, vn - 462, fp - 88, fn - 87
Accuracy: 0.8, Precision: 0.74, Recall: 0.74, F1 Score: 0.74


## Configuración 2

In [318]:
# Conseguir los datos de la segunda configuración
df2 = pd.read_csv("train_transformed.csv")
# Conseguir las variables independientes
x = df2.drop("Survived", axis=1)
# Conseguir la variable dependiente
y = df2["Survived"].tolist()

# Dividimos los datos en 70% (train) y 30% (test)
x_train, x_test, y_train, y_test = train_test_split(x, y, random_state=0, test_size=0.3)

In [319]:
# Crear modelo
model2 = LogisticRegression(independent=x_train, dependent=y_train, num_iters=300000, learning_rate=0.1)
# Entrenamos al modelo
model2.fit()

Thetas finales: [ 33.03852825  -5.42859445  -0.32340685 -16.44406017  -2.39520392
   0.99601571  -1.46541217]


### Hacemos pruebas de la configuración 2 con una matriz de confusión

In [320]:
print("Configuración 2")
predictions = model2.predict(x_test)
model2.evaluate(predictions, y_test)

Configuración 2
Model: vp - 73, vn - 143, fp - 27, fn - 25
Accuracy: 0.81, Precision: 0.73, Recall: 0.74, F1 Score: 0.73
